## Installations and imports

In [1]:
!pip install -q -U "torch==2.1.2" tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.


In [2]:
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00


In [3]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [7]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.1.2+cu121


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


## Data Preperation

In [57]:
#Data Preperations

filename = "/IMDB_Dataset.csv"


# df = pd.read_csv(filename,names=["reviews", "sentiment"],encoding="utf-8", encoding_errors="replace")

df = pd.read_csv(filename, encoding="utf-8", encoding_errors="replace")

# data = pd.read_csv("IMDB_Dataset.csv")
sentiment_counts = df['sentiment'].value_counts()

# Print the counts
print("Sentiment counts:")
print(sentiment_counts)


print(df.head())

# Split the dataset into train, test, and evaluation sets
X_train = list()
X_test = list()
y_test = list()
for sentiment in ["positive", "negative"]:  # Iterate over sentiment categories
    sentiment_df = df[df.sentiment == sentiment]
    print(f"Number of samples for sentiment category '{sentiment}': {len(sentiment_df)}")  # Print length of DataFrame

    if len(sentiment_df) < 2:
        raise ValueError(f"Not enough samples for sentiment category '{sentiment}'")

    train, test = train_test_split(sentiment_df,
                                   test_size=0.05,  # 5% of samples for testing
                                   random_state=42)
    X_train.append(train)
    X_test.append(test)
    y_test.append(test["sentiment"])

X_train = pd.concat(X_train).sample(frac=1, random_state=10)  # Shuffle the DataFrame
X_test = pd.concat(X_test)



# eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]  # Get evaluation indices
# X_eval = df[df.index.isin(eval_idx)]  # Get evaluation DataFrame
# X_eval = (X_eval.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(n=50, random_state=10, replace=True)))  # Sample 50 samples from each sentiment category
# X_train = X_train.reset_index(drop=True)  # Reset index of training DataFrame

Sentiment counts:
sentiment
positive    25000
negative    25000
Name: count, dtype: int64
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Number of samples for sentiment category 'positive': 25000
Number of samples for sentiment category 'negative': 25000


In [59]:
eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]  # Get evaluation indices
X_eval = df[df.index.isin(eval_idx)]  # Get evaluation DataFrame
X_eval = (X_eval.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(n=50, random_state=10, replace=True)))  # Sample 50 samples from each sentiment category
X_train = X_train.reset_index(drop=True)  # Reset index of training DataFrame

In [60]:
print(X_test)

                                                  review sentiment
13886  I don't know how or why this film has a meager...  positive
48027  For a long time it seemed like all the good Ca...  positive
19536  Terry Gilliam's and David Peoples' teamed up t...  positive
27232  What is there to say about an anti-establishme...  positive
28001  This movie was made only 48 years after the en...  positive
...                                                  ...       ...
14279  Dr. Seuss would sure be mad right now if he wa...  negative
14181  OK, so I gotta start this review by saying i w...  negative
28045  If you're watching this without an inkling of ...  negative
41296  This movie truly shows the farce and hypocrisy...  negative
31710  This movie sucks so bad. Its funny to see what...  negative

[2500 rows x 2 columns]


In [61]:

def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the movie review,
            determine if it is positive, or negative, and return the answer as
            the corresponding sentiment label "positive" or "negative".

            [{data_point["review"]}] = {data_point["sentiment"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the movie,
            determine if it is positive, or negative, and return the answer as
            the corresponding sentiment label "positive" or "negative".

            [{data_point["review"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["review"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), columns=["review"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["review"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

### Create a function to evaluate the results from the fine-tuned sentiment model.

In [62]:
def evaluate(y_true, y_pred):

    mapping = {'positive': 1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)


In [63]:
# model_name = "mistralai/Mistral-7B-v0.1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",device_map=device,
#     torch_dtype=compute_dtype,
#     quantization_config=bnb_config,)

model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mistral-7B-v0.1',
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1',
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [64]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["review"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")

        else:
            y_pred.append("none")
    return y_pred

y_pred = predict(test, model, tokenizer)


evaluate(y_true, y_pred)

100%|██████████| 2500/2500 [21:56<00:00,  1.90it/s]

Accuracy: 0.654
Accuracy for label 0: 0.325
Accuracy for label 1: 0.982

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.32      0.48      1250
           1       0.59      0.98      0.74      1250

    accuracy                           0.65      2500
   macro avg       0.77      0.65      0.61      2500
weighted avg       0.77      0.65      0.61      2500


Confusion Matrix:
[[ 406  844    0]
 [  22 1228    0]
 [   0    0    0]]


In [24]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["review"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")

        else:
            y_pred.append("none")
    return y_pred

y_pred = predict(test, model, tokenizer)


evaluate(y_true, y_pred)

100%|██████████| 2500/2500 [22:12<00:00,  1.88it/s]

Accuracy: 0.833
Accuracy for label 0: 0.880
Accuracy for label 1: 0.786

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.88      0.84      1250
           1       0.87      0.79      0.82      1250

    accuracy                           0.83      2500
   macro avg       0.84      0.83      0.83      2500
weighted avg       0.84      0.83      0.83      2500


Confusion Matrix:
[[1100  150    0]
 [ 268  982    0]
 [   0    0    0]]
